In [ ]:
!pip install azure-search-documents



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\ralexiou\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import json

# Your setup
endpoint = "https://search-letch.search.windows.net"
index_name = "default"
api_key = API_KEY


client = SearchClient(
    endpoint=endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(api_key)
)

results = client.search("Residential Contract of Sale", top=1)


for result in results:
    raw_payload = result.get("payload", "{}")
    payload = json.loads(raw_payload)
    file = payload.get("file", "Unknown file")
    text = payload.get("text", "[No text found]")
    tags = result.get("tags", [])
    
    print(f"📄 {file}\n\n{text[:500]}...\n")  # print first 500 chars of text



📄 Fabrice_Purchase Agreement_HandWritten.pdf

ENVIRONMENTAL MATTERS: Buyer is advised that the presence of wetlands, toxic substances, including asbestos and wastes or other environmental hazards or the presence of a threatened or endangered species or its habitat may affect Buyer's intended use of the Property. If Buyer is concerned about these matters, an addendum promulgated by TREC or required by the parties should be used.

H. SELLER'S DISCLOSURE:

(1) Seller :unselected: is
 :selected:
is not aware of any flooding of the Property whic...



In [ ]:
import asyncio
import logging


from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

import importlib
import KnowledgeBasePlugin 
importlib.reload(KnowledgeBasePlugin)


# IMPORT PLUGINS
from LightsPlugin import LightsPlugin
from KnowledgeBasePlugin import KnowledgeBasePlugin


In [ ]:
# Initialize Kernel
kernel = Kernel()


# Set up Azure OpenAI completion service
chat_completion = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    api_key=API_KEY,  
    base_url="https://aoai-devoxx2025.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"
)

# Add the service to the kernel
kernel.add_service(chat_completion)

# Logging setup
setup_logging()
logging.getLogger("kernel").setLevel(logging.DEBUG)


In [ ]:
# Add the Lights plugin (it exposes semantic functions like turn_on, turn_off, etc.)
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

# Remove previous plugin instance if needed
if "KnowledgeBase" in kernel.plugins:
    del kernel.plugins["KnowledgeBase"]


# Add the KnowledgeBase plugin (it exposes semantic functions like get_answer, get_summary, etc.)
kernel.add_plugin(
    KnowledgeBasePlugin(endpoint="https://search-letch.search.windows.net", index_name="default", api_key=API_KEY),
    plugin_name="KnowledgeBase",
)

for plugin_name, plugin in kernel.plugins.items():
    print(f"\n🧩 Plugin: {plugin_name}")
    for func in plugin:
        print(f"  └─ {func.name}")




🧩 Plugin: Lights
  └─ change_state
  └─ get_lights

🧩 Plugin: KnowledgeBase
  └─ ask_with_context
  └─ search_docs


In [135]:
# Set up execution settings
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create conversation history
history = ChatHistory()

async def chat_with_assistant(user_input: str):
    history.add_user_message(user_input)
    
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    print("Assistant >", result)
    
    history.add_message(result)


In [43]:
await chat_with_assistant("Can you show me the current state of all the lights?")


Assistant > Here is the current state of all the lights:

1. **Table Lamp**: Off
2. **Porch Light**: Off
3. **Chandelier**: Off


In [125]:
await chat_with_assistant("Use the knowledge base to explain what is a Residential Contract of Sale.")


Assistant > A **Residential Contract of Sale** is a legally binding agreement that facilitates the sale and purchase of residential real estate, typically involving single-family homes. Here are key aspects of such contracts based on the provided documents:

1. **Parties Involved**: The contract identifies the seller and buyer involved in the transaction. For example, in a specific Maryland contract, the seller is an individual (Anne Patel) and the buyer (Oscar Ward).

2. **Property Description**: The contract will include details about the property being sold, including its address and specifics about the property type.

3. **Financial Terms**: This includes the purchase price, deposit amounts, financing terms (such as loan amounts and interest rates), and payment details, including when payments are due.

4. **Contingencies**: Many contracts include conditions that must be met for the sale to proceed. Common contingencies involve financing approval, home inspections, and appraisal re

In [136]:
await chat_with_assistant("Show me documents about seller disclosures.")


Assistant > Here are some documents related to seller disclosures:

1. **[Manmaya-handwritten-single-family-home-contract.pdf](#)**  
   This document is a contract between the seller, Noel Belisle, and the buyer, Manmaya Panigrahi, for a property located in Los Angeles, CA. It details the sales price of $500,000, earnest money requirements, and the seller's obligations regarding title policy and survey. The contract includes provisions for repairs, disclosures, and remedies for potential defaults.

2. **[Oscar-Ward-Real-Estate-Contract.pdf](#)**  
   This document discusses obligations related to lead poisoning prevention in Maryland. It states that any residential dwelling constructed prior to 1950 must be registered with the Maryland Department of the Environment. The contract includes details on disclosures required for properties built before 1979 and mentions several addenda that may be attached, including property disclosures and inspections.

3. **[Eric-handwritten-new-construc